In [ ]:
from copy import deepcopy

import numpy as np
from numpy import asarray, sqrt, eye, abs as npabs, kron, pi, ndarray, append
from matplotlib.pyplot import subplots, show

from pytreenet.core import Node
from pytreenet.ttns import TreeTensorNetworkState
from pytreenet.operators import TensorProduct, Hamiltonian, pauli_matrices
from pytreenet.ttno import TreeTensorNetworkOperator

from pytreenet.time_evolution.bug import BUG
from pytreenet.time_evolution.exact_time_evolution import ExactTimeEvolution
from pytreenet.time_evolution.ttn_time_evolution import TTNTimeEvolutionConfig
from pytreenet.util.tensor_splitting import SVDParameters

In [ ]:
# Question: How to get the dimension of the node?
dim = 2
identity = eye(dim)

time_step_size = 0.01
final_time = 3.2

X, Y, Z = pauli_matrices()


In [ ]:
def generate_z_operator(node_id):
    sq_op_id = "SQ_Operator"
    sq_z_op = {sq_op_id: TensorProduct({node_id: Z})}
    return sq_z_op

## Single Qubit Gate Simulation

In [ ]:
def apply_single_qubit_gate(ttns, node_id, gate_hamiltonian):
    # Bug - USE THIS ONE
    term = TensorProduct({node_id: "gate_to_apply"})
    conv_dict = {"I2": identity, "gate_to_apply": gate_hamiltonian}
    hamiltonian = Hamiltonian(term, conversion_dictionary=conv_dict)
    ttno = TreeTensorNetworkOperator.from_hamiltonian(hamiltonian, ttns)

    z_operator = generate_z_operator(node_id)
    bug = BUG(ttns, ttno, time_step_size, final_time, z_operator)

    bug.run()

    # Exact time evolution
    ref_operator = {"SQ_Operator": Z}
    exact_evo = ExactTimeEvolution(
        ttns, gate_hamiltonian, time_step_size, final_time, ref_operator
    )
    exact_evo.run()
    
    return ttns
    

## Two Qubit Gate Simulation

In [ ]:
def apply_two_qubit_gate(ttns, control_id, target_id, gate_name):
    term = None
    control_op = None
    target_op = None
    conv_dict = None
    hamiltonian = None
    ttno = None
    match gate_name:
        case "CNOT":
            term = TensorProduct({control_id: "control_op", target_id: "target_op"})
            control_op = eye(dim) - Z
            target_op = eye(dim) - X
            conv_dict = {
                "I2": identity,
                "control_op": control_op,
                "target_op": target_op,
            }
            hamiltonian = Hamiltonian(term, conversion_dictionary=conv_dict)
        case "SWAP":
            term = [
                TensorProduct({control_id: "X", target_id: "X"}),
                TensorProduct({control_id: "Y", target_id: "Y"}),
                TensorProduct({control_id: "Z", target_id: "Z"}),
                TensorProduct({control_id: "I2", target_id: "I2"}),
            ]
            conversion_dict = {"X": X, "Y": Y, "Z": Z, "I2": identity}
            hamiltonian = Hamiltonian(
                term,
                conversion_dictionary=conversion_dict,
            )

    tq_ops = {
        control_id: TensorProduct({control_id: Z}),
        target_id: TensorProduct({target_id: Z}),
    }

    ttno = TreeTensorNetworkOperator.from_hamiltonian(hamiltonian, ttns)
    config = TTNTimeEvolutionConfig(record_bond_dim=True)
    final_time = 2 * 3.2
    bug = BUG(ttns, ttno, time_step_size, final_time, tq_ops, config=config)

    bug.run()
    return ttns

: 

In [ ]:
def apply_gate(ttns, gate_name, *qubit_ids):
    X, Y, Z = pauli_matrices()
    H = (1 / np.sqrt(2)) * (X + Z)
    one_qubit_gate_map = {"X": X, "Y": Y, "Z": Z, "H": H}
    two_qubit_gate = ["CNOT", "SWAP"]
    if gate_name in one_qubit_gate_map:
        node_id = qubit_ids[0]
        gate_hamiltonian = one_qubit_gate_map[gate_name]
        ttns.apply_single_qubit_gate(ttns, node_id, gate_hamiltonian)
    elif gate_name in two_qubit_gate:
        control_id, target_id = qubit_ids
        ttns.apply_two_qubit_gate(ttns, control_id, target_id, gate_name)
    else:
        raise ValueError(f"Unknown gate: {gate_name}")
    return ttns

## Discrete Time Evolution Example

In [ ]:
def discrete_time_evolution(ttns, gate_sequence):
    for gate in gate_sequence:
        gate_name = gate[0]
        qubit_ids = gate[1:]
        ttns = apply_gate(ttns, gate_name, *qubit_ids)
    return ttns


ttns = create_binary_tree(num_nodes=9)
gate_sequence = [("X", "0"), ("H", "2"), ("CNOT", "1", "2")]
final_state = discrete_time_evolution(ttns, gate_sequence)

# Discrete Time Evoltion
### Using the QuantumGate class

In [ ]:
from pytreenet.quantum_gates.QuantumGate import CNOTGate, HadamardGate, SWAPGate, XGate, YGate, ZGate


from pytreenet.quantum_gates.QuantumGate import CNOTGate, HadamardGate, SWAPGate, XGate, YGate, ZGate


def apply_gate(ttns, gate_name, *qubit_ids):
    """
    Apply the given quantum gate to the TTN state.
    
    Args:
        ttns: TreeTensorNetworkState (the quantum state).
        gate_name: Name of the gate ("X", "H", "CNOT", etc.).
        qubit_ids: IDs of the qubits on which the gate acts.
    
    Returns:
        Updated TTN state after applying the gate.
    """
    gate_classes = {
        "X": XGate(),
        "Y": YGate(),
        "Z": ZGate(),
        "H": HadamardGate(),
        "CNOT": CNOTGate(),
        "SWAP": SWAPGate(),
    }

    gate = gate_classes.get(gate_name)

    if not gate:
        raise ValueError(f"Unknown gate: {gate_name}")

    if gate_name == "CNOT":
        if len(qubit_ids) < 2:
            raise ValueError("CNOT gate requires at least one control and one target qubit.")
        control_id = qubit_ids[0]
        target_ids = qubit_ids[1:]
        gate.apply_gate(ttns, control_id, *target_ids)
    elif len(qubit_ids) == 1:
        gate.apply_gate(ttns, qubit_ids[0])
    elif len(qubit_ids) == 2:
        gate.apply_gate(ttns, qubit_ids[0], qubit_ids[1])
    else:
        raise ValueError("Unsupported number of qubits for the gate.")
    
    return ttns


In [ ]:
def discrete_time_evolution(ttns, gate_sequence):
    """
    Apply a sequence of quantum gates to the TTN state.
    
    Args:
        ttns: TreeTensorNetworkState (the quantum state).
        gate_sequence: List of tuples, where each tuple specifies a gate name 
                       and the qubits it acts on.
    
    Returns:
        Updated TTN state after the sequence of gates is applied.
    """
    for gate in gate_sequence:
        gate_name = gate[0]
        qubit_ids = gate[1:]
        ttns = apply_gate(ttns, gate_name, *qubit_ids)
    return ttns

In [ ]:
gate_sequence = [
    ("CNOT", "0", "1", "2", "3"),
    ("X", "1"),
    ("Y", "2"),
    ("Z", "3"),
    ("H", "0"),
    ("SWAP", "1", "2"),
]


# ttns = create_binary_tree(num_nodes=4)
# final_state = discrete_time_evolution(ttns, gate_sequence)
# print(final_state)

NameError: name 'create_binary_tree' is not defined